In [49]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from config import g_key

In [50]:
newdf3 = "../WeatherPy/csvdata/weatherdata.csv"
newdf4 = pd.read_csv(newdf3)
newdf4.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windiness,Country
0,Shiyuan,35.75,102.81,26.83,58,25,5.95,CN
1,Garoua,9.30,13.40,67.87,14,100,2.35,CM
2,Tiradentes,-21.11,-44.18,63.75,94,41,8.86,BR
3,Beihuaidian,39.26,117.56,44.60,26,0,4.47,CN
4,Maneromango,-7.20,38.78,78.80,94,40,1.68,TZ


In [60]:
gmaps.configure(api_key=g_key)

In [64]:
newdf5 = newdf4.dropna()

In [65]:
Humidity = newdf5["Humidity"].astype(float)

In [58]:
locations = newdf5[["Latitude", "Longitude"]].astype(float)

In [59]:
fig = gmaps.figure()

In [61]:
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max(newdf5["Humidity"]),point_radius = 1)

In [62]:
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [70]:
#Creating List of Cities with Ideal conditions
idealdf = newdf5
#temp less than 90 and more than 70
idealdf = idealdf.loc[(idealdf["Temperature"] <90) & (idealdf["Temperature"] > 70)]
# windiness less than 10 mph.
idealdf = idealdf.loc[idealdf["Windiness"] < 10]
# No clouds.
idealdf = idealdf.loc[idealdf["Cloudiness"] <=0]

idealdf2 = idealdf.dropna()
idealdf2

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windiness,Country
40,Le Blanc-Mesnil,17.33,102.83,86.43,66,0,4.70,TH
87,Castel San Giovanni,-34.60,-58.49,72.81,88,0,9.17,AR
89,Villa del Parque,11.10,79.67,80.06,73,0,5.26,IN
99,Junxi,17.30,97.02,88.29,35,0,4.76,MM
115,Saint-Coulomb,-34.17,-58.96,73.06,88,0,5.82,AR
144,Kaiyuan,7.85,123.83,82.36,56,0,6.26,PH
216,Pegões,19.30,109.24,84.63,58,0,8.14,CN
313,Bargstedt,-20.60,-49.30,82.40,51,0,6.93,BR
375,Sinor,-34.49,-59.08,72.16,88,0,5.82,AR
387,Zitlaltépec,7.80,124.73,81.81,39,0,3.38,PH


In [78]:
#Finding Hotels nearby
hotels = idealdf2
params = {"radius": 500,"types": "lodging","key": g_key}

for index, row in hotels.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    params["location"] = f"{lat},{lon}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f" Hotels near {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotels.loc[index, 'Hotel Name'] = results[0]['name']    
    except (KeyError, IndexError):
        print("error")
        
    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

 Hotels near Le Blanc-Mesnil.
Closest hotel is ที่ทำการผู้ใหญ่บ้าน.
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
 Hotels near Castel San Giovanni.
Closest hotel is Y Alojamiento.
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
 Hotels near Villa del Parque.
Closest hotel is Madurai Muniyandi vilas.
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
 Hotels near Junxi.
error
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
 Hotels near Saint-Coulomb.
Closest hotel is Hotel Morada Suites.
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
 Hotels near Kaiyuan.
error
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
 Hotels near Pegões.
Closest hotel is An'an Hostel.
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
 Hotels near Bargstedt.
error
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
 Hotels near Sinor.
error
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
 Hotels near Zitlaltépec.
error
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


In [79]:
# Marking Hotels on map
locations = hotels[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)
info_box_template = """
<dt>Country</dt><dd>{Country}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotels.iterrows()]
hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)',stroke_color='rgba(0, 0, 150, 0.4)', scale=1,info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))